# Cleaning of shark attacks data set

## Objetive

The main purpose of this project is double. In one hand, we want to know what shark species is la que más está registrada en los ataques. In the other hand, we want to determine the profile of the people who has provoked the incidents the most (nationality, activity, sex and age).

In [ ]:
#Imports
import pandas as pd
import numpy as np

In [8]:
#Import data
data = pd.read_csv('../data/attacks.csv', encoding='latin-1')
print(data.shape)
data.head()
data.columns


(25723, 24)


Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

Our first task is to subset the initial dataset to keep only the variables that we are going to use:
* Type
* Country
* Activity
* Sex
* Age
* Species

In [17]:
# Subset the dataframe with the needed columns
df_subset = data[["Type", "Country", "Activity", "Sex ", "Age", "Species "]]

# Rename the Sex and Species columns to remove the last space and avoid problems in the future...
df_subset.rename(columns={"Sex ": "Sex", "Species ": "Species"})
df_subset.head()

,Type,Country,Activity,Sex,Age,Species
0,Boating,USA,Paddling,F,57,White shark
1,Unprovoked,USA,Standing,F,11,NaN
2,Invalid,USA,Surfing,M,48,NaN
3,Unprovoked,AUSTRALIA,Surfing,M,NaN,2 m shark
4,Provoked,MEXICO,Free diving,M,NaN,"Tiger shark, 3m"


Counting the NULLs and NaN's 


In [46]:
size = df_subset.shape

nulls = df_subset.isnull().sum()
nulls


Type        19425
Country     19471
Activity    19965
Sex         19986
Age         22252
Species     22259
dtype: int64

In [45]:
num_nas = df_subset.isna().sum()
num_nas

Type        19425
Country     19471
Activity    19965
Sex         19986
Age         22252
Species     22259
dtype: int64

Because of the huge amount of Null values in shark species, we are going to split the data into two parts, one for analizar las especies y otro para los humanos

In [ ]:
Sharks attacks provoked by human by country